In [10]:
import sys
import os
import re
import numpy as np
import torch
from torch.multiprocessing import Process, set_start_method
from functools import partial
from utils import record_stats, display_stats, distribute
from pathlib import Path 


%load_ext autoreload
%autoreload 2

In [11]:
n_cpu = 12
n_instance = 12
nodesels = ['oracle', 'gnn_trained']

problem = "FCMCNF"
normalize = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
verbose = True
on_log = False
default = True
delete = False

if delete:
    try:
        import shutil
        shutil.rmtree(os.path.join(os.path.abspath(''), 
                                       'stats'))
    except:
        ''



instances = list(Path(os.path.join(os.path.abspath(''), 
                                   f"./problem_generation/data/{problem}/test")).glob("*.lp"))
if n_instance == -1 :
    n_instance = len(instances)

instances = instances[:n_instance]

print("Evaluation")
print(f"  Problem:                    {problem}")
print(f"  n_instance/problem:         {len(instances)}")
print(f"  Nodeselectors evaluated:    {','.join(nodesels + ['default' if default else '' ])}")
print(f"  Device for GNN inference:   {device}")
print(f"  Normalize features:         {normalize}")
print("----------------")




Evaluation
  Problem:                    FCMCNF
  n_instance/problem:         12
  Nodeselectors evaluated:    oracle,gnn_trained,default
  Device for GNN inference:   cpu
  Normalize features:         True
----------------


In [12]:

processes = [  Process(name=f"worker {p}", 
                                target=partial(record_stats,
                                                nodesels=nodesels,
                                                instances=instances[p1:p2], 
                                                problem=problem,
                                                device=torch.device(device),
                                                normalize=normalize,
                                                verbose=verbose,
                                                default=default))
                for p,(p1,p2) in enumerate(distribute(n_instance, n_cpu)) ]  


try:
    set_start_method('spawn')
except RuntimeError:
    ''

a = list(map(lambda p: p.start(), processes)) #run processes
b = list(map(lambda p: p.join(), processes)) #join processes

min_n = min([ int( str(instance).split('=')[1].split('_')[0] )  for instance in instances ] )
max_n = max([ int( str(instance).split('=')[1].split('_')[0] )  for instance in instances ] )

display_stats(problem, nodesels, instances, min_n, max_n, default=default)

<All keys matched successfully>
------------------------------------------
   |----Solving:  FCMCNF
   |----Instance: /local_workspace/labaabde/learn2selectnodes/problem_generation/data/FCMCNF/test/n_nodes=20_n_arcs=96_n_commodities=21_id_45.14.lp
   |----Nodesel: gnn_trained
<All keys matched successfully>
------------------------------------------
   |----Solving:  FCMCNF
   |----Instance: /local_workspace/labaabde/learn2selectnodes/problem_generation/data/FCMCNF/test/n_nodes=20_n_arcs=84_n_commodities=28_id_35.74.lp
   |----Nodesel: gnn_trained
<All keys matched successfully>
------------------------------------------
   |----Solving:  FCMCNF
   |----Instance: /local_workspace/labaabde/learn2selectnodes/problem_generation/data/FCMCNF/test/n_nodes=20_n_arcs=80_n_commodities=27_id_96.00.lp
   |----Nodesel: gnn_trained
<All keys matched successfully>
------------------------------------------
   |----Solving:  FCMCNF
   |----Instance: /local_workspace/labaabde/learn2selectnodes/problem